# Review of Repo Operations

## Primary

[OFR](https://www.financialresearch.gov/)

* Hedge Funds
  - size - gross notional exposure by asset class (USD)
  - size - gross notional exposure by strategy class (USD)
  - leverage - borrowing by type (USD)
  - leverage - long and short repo exposures (USD)
* UM Money Market Fund Monitor
  - Fed Res Repo facility total utilization and MMF's participation
* Short-term Funding Monitor
  - Repo transaction volume by venue
  - Tri-party repo transation volumen collateral type
  - Money market fund investment holding by asset type
  - US Repo Mkts - Tri-party repo transaction volume collateral type
  - US Repo Mkts - DVP Service Average Rate: Total (final)
  - US Repo Mkts - GCF Repo Service Average Rate: Total (final)
  - US Repo Mkts - Triparty Average Rate: Total (final)
  - Repo tranaction volumes by venue
  - Reference rates: Effective Federal Funds Rate
  - Primary Dealer Statistics: Primary Dealer Aggregate Fails to Deliver (total)

[Federal Reserve NY](https://www.newyorkfed.org/markets/data-hub)

* Fed Res Repo Ops - Submitted, Accepted, Rate
* Fed Res Repo Ops - Submitted, Accepted, Rate (by security type Treaury, Agency, MBS)
* Fed Res Balance Sheet (H.4.1)
  - Reserves
  - US Treasuries
  - Agency Securities
  - MBS
* Fed Res Balance Sheet
  - Loans - Primary Credit
  - Loans - Secondary Credit
* Fed Res Balance Sheet
  - Central Bank Liquidity Swaps

[DTCC](dtcc.com/charts/membership)

* Spaonsored DVP and Sponsored GC Activity
* Sponsored Repo and Sponsored Reverse Repo Activity



## Additional

[US Treasury Month Debt](https://fiscaldata.treasury.gov/datasets/monthly-statement-public-debt/summary-of-treasury-securities-outstanding)

* Bills outstanding
* ???

[SIFMA](https://www.sifma.org/resources/research/statistics/us-treasury-securities-statistics/#:~:text=Issuance%20(as%20of%20February)%20$5.2,%2C%20+6.8%25%20Y/Y)

* US Treasuries Security Issuance


FRED

* Money Market Funds; Total Financial Assets, Level (MMMFFAQ027S)
* Overnight Reverse Repurchase Agreements: Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTSYD)
* Liabilities and Capital: Liabilities: Deposits with F.R. Banks, Other Than Reserve Balances: U.S. Treasury, General Account: Week Average (WTREGEN)
* Liabilities and Capital: Other Factors Draining Reserve Balances: Reserve Balances with Federal Reserve Banks: Wednesday Level (WRBWFRBL)
* SOFR
  - Secured Overnight Financing Rate (SOFR)
  - Secured Overnight Financing Rate: 99th Percentile (SOFR99, SOFR75, SOFR25, SOFR1
  - Secured Overnight Financing Volume (SOFRVOL)
* FedFunds
  - Federal Funds Effective Rate (FEDFUNDS)
  - Effective Federal Funds Volume (EFFRVOL)
* Overnight
  - Overnight Bank Funding Volume (OBFRVOL)
* Calculated
  - SOFR Volume / Liabilities and Capital: Other Factors Draining Reserve Balances

Yahoo

* SP500
* Price gold
* Price silver
* calc: sp500 / gold
* calc: gold / silver


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#project
from macroecon_wrappy import (
    Auth,
    Metric,
    Measure,
    Epoch,
    plots
)
from macroecon_wrappy.adapters import (
    FredApi, 
    TreasuryFiscal
)
from macroecon_wrappy.extractors import (
    NberExtract
)


#external
from fredapi import Fred
#from treasury.client import FederalTreasuryClient


#system
from pathlib import Path

In [3]:
secrets_path = Path('/workspaces/API-macroecon-wrappy/SECRETS.yaml')
cache_path = Path('/workspaces/API-macroecon-wrappy/tests/tmp/')
auth = Auth(secrets_path, cache_path)
auth.load_secrets()

True

In [4]:
#nber
NberExtract.set_config(auth)
name = 'recessions'
recessions = NberExtract.get_data(name)

first two columns will be renamed: ['start', 'end', 'name']


## Reverse Repos and T-bills

In [ ]:
#fred
#source: https://fred.stlouisfed.org/source?soid=23
# get this api: https://fiscaldata.treasury.gov/datasets/monthly-statement-public-debt/summary-of-treasury-securities-outstanding
# compare with sifma: https://www.sifma.org/resources/research/statistics/us-treasury-securities-statistics/#:~:text=SIFMA%20Research%20tracks%20U.S.%20Treasury,%2C%20+6.8%25%20Y/Y
FredApi.set_wrapper(auth, Fred)
mmftfa = FredApi.get_data('MMMFFAQ027S')
tb1yr = FredApi.get_data('DTB1YR')                  #rate - not what we want
treasfinancing = FredApi.get_data('MTSMFBP133FMS')  #amount of all debt - we actually want Tbills from fiscal site: 

In [ ]:
#TreasuryFiscal.set_wrapper(auth, FederalTreasuryClient)
#debt_outstanding = TreasuryFiscal.get_data('other_data-historical_debt_outstanding')
#debt_outstanding = TreasuryFiscal.get_data('public_debt_instruments-details_of_securities_outstanding')
#measure = Measure([mmftfa, debt_outstanding], recessions)

Money Markets are Reverse Repos and T-bills.  MM Funds invest there for a little interest.  If there is less T-bills then more Repos are needed to fullfil the MM needs.

In [13]:
#TODO:add to plot
import altair as alt

base = alt.Chart(measure.df(interpolated=True).reset_index(), width=600, height=200).encode(x='timestamp:T')
line1 = base.mark_line(stroke='blue').encode(alt.Y('col-0'))#.transform_filter('isValid(datum.line)')
line2 = base.mark_line(stroke='orange').encode(alt.Y('col-1'))
(line1 + line2).resolve_scale(y = 'independent')

alt.LayerChart(...)

## Gold and the Stock Market

The relationship between gold and the sp500 is high.

In [23]:
FredApi.set_wrapper(auth, Fred)
IQ12260 = FredApi.get_data('IQ12260')
SP500 = FredApi.get_data('SP500')
measure = Measure([SP500, IQ12260], recessions)
measure.df().head(3)
measure


        Metrics: 
            None(col-0)  None(col-1)
timestamp                           
2025-03-21      5667.56          NaN
2025-03-20      5662.89          NaN
2025-03-19      5675.29          NaN
2025-03-18      5614.66          NaN
2025-03-17      5675.12          NaN
        Cycle: 
       start        end name
1 1857-06-01 1858-12-01     
2 1860-10-01 1861-06-01     
3 1865-04-01 1867-12-01     
4 1869-06-01 1870-12-01     
5 1873-10-01 1879-03-01      
        

In [9]:
#TODO:add to plot
import altair as alt

base = alt.Chart(measure.df(interpolated=True).reset_index(), width=600, height=200).encode(x='timestamp:T')
line1 = base.mark_line(stroke='blue').encode(alt.Y('col-0'))#.transform_filter('isValid(datum.line)')
line2 = base.mark_line(stroke='orange').encode(alt.Y('col-1'))
(line1 + line2).resolve_scale(y = 'independent')

alt.LayerChart(...)

In [28]:
tmp = measure.df(interpolated=True).reset_index()
tmp['col-2'] = tmp['col-0'] / tmp['col-1'] 
tmp = tmp[tmp.timestamp > dt.datetime(2014, 1, 1)]
tmp

,timestamp,col-0,col-1,col-2
0,2025-03-21,5667.56,951.9,5.953945
1,2025-03-20,5662.89,951.9,5.949039
2,2025-03-19,5675.29,951.9,5.962065
3,2025-03-18,5614.66,951.9,5.898372
4,2025-03-17,5675.12,951.9,5.961887
...,...,...,...,...
2651,2014-06-01,NaN,435.9,NaN
2652,2014-05-01,NaN,453.5,NaN
2653,2014-04-01,NaN,466.1,NaN
2654,2014-03-01,NaN,455.1,NaN


In [29]:
#TODO:add to plot
import altair as alt
import pandas as pd

base = alt.Chart(tmp, width=600, height=200).encode(x='timestamp:T')
line1 = base.mark_line(stroke='blue').encode(alt.Y('col-2'))#.transform_filter('isValid(datum.line)')
line1

alt.Chart(...)

In [8]:
from macroecon_wrappy import plots
charts = plots.graph_ts_js(measure, cols=['col-0', 'col-1'], recession_bars=False, log_scale=None)
charts

alt.VConcatChart(...)

## The TGA and Bank Reserves

In [10]:
FredApi.set_wrapper(auth, Fred)
D2WLTGAL = FredApi.get_data('D2WLTGAL')
WRESBAL = FredApi.get_data('WRESBAL')
measure = Measure([D2WLTGAL, WRESBAL], recessions)
measure.df().head(3)
measure


        Metrics: 
            None(col-0)  None(col-1)
timestamp                           
2025-03-19     415799.0     3467.789
2025-03-12     450713.0     3400.235
2025-03-05     522818.0     3312.625
2025-02-26     568550.0     3327.249
2025-02-19     738938.0     3270.082
        Cycle: 
       start        end name
1 1857-06-01 1858-12-01     
2 1860-10-01 1861-06-01     
3 1865-04-01 1867-12-01     
4 1869-06-01 1870-12-01     
5 1873-10-01 1879-03-01      
        

In [15]:
import datetime as dt

tmp = measure.df(interpolated=True).reset_index()
tmp = tmp[tmp.timestamp > dt.datetime(2024, 1, 2)]
tmp

,timestamp,col-0,col-1
0,2025-03-19,415799.0,3467.789
1,2025-03-12,450713.0,3400.235
2,2025-03-05,522818.0,3312.625
3,2025-02-26,568550.0,3327.249
4,2025-02-19,738938.0,3270.082
...,...,...,...
59,2024-01-31,865481.0,3526.230
60,2024-01-24,815159.0,3526.683
61,2024-01-17,773544.0,3611.719
62,2024-01-10,727856.0,3498.748


In [16]:
import altair as alt

base = alt.Chart(tmp, width=600, height=200).encode(x='timestamp:T')
line1 = base.mark_line(stroke='blue').encode(alt.Y('col-0'))#.transform_filter('isValid(datum.line)')
line2 = base.mark_line(stroke='orange').encode(alt.Y('col-1'))
(line1 + line2).resolve_scale(y = 'independent')

alt.LayerChart(...)